<p><b><center>Investigating Smooth-Particle Hydrodynamics (SPH) with Python</center></b></p>

<u>Table of Contents</u>
1. Introduction
2. Modelling
3. The $n$-body model
4. The dam break model
5. Extensions
6. Conclusion
7. References
8. Appendices

# Introduction
## Fluid dynamics: why bother?
[Fluid dynamics](https://en.wikipedia.org/wiki/Fluid_dynamics) is, broadly speaking, the study of how fluids flow. Understanding how fluids flow is important for many specializations in engineering. Here are some interesting applications of fluid dynamics:
1. [Biomedical Engineering](https://pubmed.ncbi.nlm.nih.gov/34476751/): the design of medical equipment such as vascular stents relies on a good understanding of how blood flows in the body.
2. [Disaster Protection](https://link.springer.com/chapter/10.1007/978-3-211-69356-8_1): tsunami forecasting and risk evaluation requires a good understanding of how the oceans move.
3. [Industrial Design](https://www.dynamiq-eng.co.uk/pages/Fluid_mechanics_dynamics.php): building safe, efficient, and functioning equipment needs a good understanding of how "fluids" such as heat and pressure change over time.
4. [Astrophysics](https://arxiv.org/abs/1109.2219): somewhat surprisingly, we can model space as a fluid as well.

## Fluid dynamics: how to implement?
As with anything in science, we first begin our study of fluid dynamics by learning how to model it. Fluids are typically modelled as _continuous_, meaning that we treat them as a continuous mass, rather than a set of discrete particles. Consider a mole of water - is it easier to think of it as $18ml$ of water or $\sim 6\times10^{23}$ water molecules interacting with each other?

In our simplified models, these bodies of water then obey several assumptions relating quantities such as their pressure, density, volume, and mass. We briefly list some assumptions, taken from [Wikipedia](https://en.wikipedia.org/wiki/Fluid_mechanics):
1. Conservation of mass
2. Conservation of energy
3. Conservation of momentum
4. The continuum assumption

We can explain what these assumptions mean using a simple bowl of water taken from [Shutterstock](https://www.shutterstock.com/image-vector/water-bowl-illustration-vector-cartoon-260nw-2210641359.jpg).

![](https://www.shutterstock.com/image-vector/water-bowl-illustration-vector-cartoon-260nw-2210641359.jpg)

**Conservation of mass**: the amount of water in the bowl will not change (unless we pour some water out, or pour some water in).

**Conservation of energy**: the energy of the water in the bowl will not change (assuming the bowl is an inch thick shell of $Bi_4O_4SeCl_2$, the [worst known heat conductor in the world](https://www.sciencealert.com/sustainability-win-scientists-make-material-with-the-lowest-thermal-conductivity-yet), preventing the heat energy of the water from moving to its surroundings).

**Conservation of momentum**: the total momentum of the water remains constant, assuming that the bowl is a closed system.

**The continuum assumption**: the body of water is a continuous distribution of matter, where properties such as pressure ["may vary point-by-point."](http://pillars.che.pitt.edu/student/slide.cgi?course_id=10&slide_id=5.0) A better example than our bowl of water would the ocean - the temperature of the water on the beaches of Delhi is different from the temperature of the water on the shores of Normandy, even though the oceans can be thought of as a continuous distribution of water.

With these assumptions, we can then proceed to model the various properties of the fluid using a simple set of equations. [NASA](https://www.grc.nasa.gov/www/k-12/airplane/nseqs.html) has provided a helpful graphic showing the equations we use:

![](https://www.grc.nasa.gov/www/k-12/airplane/Images/nseqs.gif)

Oh.

## Practical fluid dynamics: challenges
If we are to follow the equations in the NASA graphic, we are going to have one hell of a time trying to solve anything. Solving a single partial differential equation (PDE) is tough enough on its own, and solving several linked PDEs is very very hard. If we are to try to solve it by hand, we quickly realize that doing so is an [open problem with a bounty of a million dollars](https://en.wikipedia.org/wiki/Navier–Stokes_existence_and_smoothness).

The challenges are obvious, and seemingly insurmountable. Despite this, engineers and scientists have been successfully surmounting them for decades. How?

## Practical fluid dynamics: approaches
There is, however, an easy solution to the challenges posed above: [computational fluid dynamics (CFD)](https://www.simscale.com/docs/simwiki/cfd-computational-fluid-dynamics/what-is-cfd-computational-fluid-dynamics/). To explain CFD, we briefly return to secondary school calculus. We recall the geometric intuition behind differentiation - differentiating our function gives us the rate of change of the function, and partial differential equations are nothing but lots of differentiation.

In an ideal world, the derivative $f'(x)$ of a function $f(x)$ is given by the equation $$f'(x)=\lim_{h\to 0}\frac{f(x+h)-f(x)}{h}.$$ In our unideal world, we can set the value of $h$ to $0.005$ (an arbitrary number), and declare that to be the rate of change over time. In other words, we set our time step to $0.005$ seconds (or any other sufficiently small number that you like), and use this to approximate the complicated equations we see above.

This converts the problem from a calculus problem with differentiation and integration to a computational problem, involving millions of addition, subtraction, multiplication, and division operations. To a computer, the latter is much simpler to perform than the former.

This allows us to introduce smooth-particle hydrodynamics (SPH), a computational method that translates the fancy equations to a much uglier, but simpler to compute version.

## SPH: bow chicka wow wow
Daniel Price has published a [layman's guide](https://arxiv.org/pdf/1111.1259.pdf) to SPH. Here is a brief summary of the main points:


# Modelling Fluid Dynamics
To translate the equations of SPH into code, we first begin by clearly identifying the requirements and desired outcomes of the mathematical model. (something or other)

We



- Using a `Particle` data type
- Important variables to store
- Introducing a kernel function

# The $n$-body problem
- Problem statement
- Code implementation
- Pretty pictures
- Problems with naive implementation of $n$-body problem

# The dam break model
- Problem statement
- Code implementation
- Pretty pictures
- Explanation of why SPH is good

# Extensions
- How SPH can be extended to other fields/case studies
- Optimizations to our code

# Conclusion
- SPH good

# References
ACS citation style.

# Appendices
- Zhen Jie's explanation of the science
- Any other supplementary nonsense or further extensions of the work above.